In [2]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
import pickle
import os
import logging
import time
from IPython.core.debugger import set_trace
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F

from utils.utils import save_checkpoint, load_checkpoint, set_logger
from utils.gpu_utils import set_n_get_device

from dataset.dataset import prepare_trainset

#from model.model_unet_classify_zero import UNetResNet34
from model.model_unet_classify_zero2 import UNetResNet34
from model.deeplab_model_kaggler.lr_scheduler import LR_Scheduler

%matplotlib inline

In [3]:
######### Config the training process #########
#device = set_n_get_device("0, 1, 2, 3", data_device_id="cuda:0")#0, 1, 2, 3, IMPORTANT: data_device_id is set to free gpu for storing the model, e.g."cuda:1"
MODEL = 'UNetResNet34'#'RESNET34', 'RESNET18', 'INCEPTION_V3', 'BNINCEPTION', 'SEResnet50'
#AUX_LOGITS = True#False, only for 'INCEPTION_V3'
print('====MODEL ACHITECTURE: %s===='%MODEL)

device = set_n_get_device("0", data_device_id="cuda:0")#0, 1, 2, 3, IMPORTANT: data_device_id is set to free gpu for storing the model, e.g."cuda:1"
multi_gpu = None #[0, 1] #None

SEED = 1234#5678#4567#3456#2345#1234
debug = True# if True, load 100 samples
IMG_SIZE = (256,1600)
BATCH_SIZE = 16
NUM_WORKERS = 24
torch.cuda.manual_seed_all(SEED)

====MODEL ACHITECTURE: UNetResNet34====


## the dataset

In [5]:
train_dl, val_dl = prepare_trainset(BATCH_SIZE, NUM_WORKERS, SEED, IMG_SIZE, debug)

Count images in train/valid folder:  12568 1801
Count of trainset (for training):  1120
Count of validset (for training):  120


In [6]:
for i, (images, masks) in enumerate(train_dl):
    images = images.to(device=device, dtype=torch.float)
    #1 for non-zero-mask
    truth = (torch.sum(masks.reshape(masks.size()[0], masks.size()[1], -1), dim=2, keepdim=False)!=0).to(device=device, dtype=torch.float)
    if i==0:
        break

In [7]:
images.size(), truth.size()

(torch.Size([16, 1, 256, 1600]), torch.Size([16, 4]))

In [8]:
truth

tensor([[0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.]], device='cuda:0')

## the model

In [9]:
net = UNetResNet34(debug=True).cuda(device=device)
#net = ZeroMaskClassifier(debug=True).cuda(device=device)

#torch.cuda.set_device(0)
#torch.distributed.init_process_group(backend='nccl', world_size=4, init_method='...')
#net = DistributedDataParallel(net, device_ids=[0], output_device=0)
#torch.distributed.init_process_group(backend="nccl")

#checkpoint_path = 'checkpoint/UNetResNet34_512_v1_seed3456/best.pth.tar'
#net, _ = load_checkpoint(checkpoint_path, net)

if multi_gpu is not None:
    net = nn.DataParallel(net, device_ids=multi_gpu)

In [10]:
logit = net(images)

input:  torch.Size([16, 3, 256, 1600])
e1 torch.Size([16, 64, 128, 800])
e2 torch.Size([16, 64, 128, 800])
e3 torch.Size([16, 128, 64, 400])
e4 torch.Size([16, 256, 32, 200])
e5 torch.Size([16, 512, 16, 100])
avgpool torch.Size([16, 512, 1, 1])
reshape torch.Size([16, 512])
logit_clf torch.Size([16, 4])


In [11]:
if multi_gpu:
    loss = net.module.criterion(logit, truth)
else:
    loss = net.criterion(logit, truth)

loss

tensor(0.8676, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [14]:
if multi_gpu:
    _, metric, tn, fp, fn, tp, pos_percent = net.module.metric(logit, truth)
else:
    _, metric, tn, fp, fn, tp, pos_percent = net.metric(logit, truth)

metric, tn, fp, fn, tp, pos_percent

(0.26415, 28, 25, 11, 0, 0.390625)

In [ ]:
%%time
#label_df = pd.read_csv('data/raw/NIH/NIH external data/label_df_pneumothorax.csv').set_index('img_id')

for idx, img_id in enumerate(label_df.index):
    if idx>10000:
        break
    has_pneumothorax = label_df.loc[img_id, 'has_pneumothorax']


In [ ]:
for idx, f in enumerate(glob.glob('data/raw/NIH/NIH external data/images*/*')):
    img = plt.imread(f)
    if len(img.shape)==3:
        print(img.shape)
        break
    if idx>100:
        break
#img_path = glob.glob('data/raw/NIH/NIH external data/images*/00000001_000.png')[0]#00000003_000.png
#img = plt.imread(img_path)

In [ ]:
img.shape

In [ ]:
#plt.imshow(img)

## predict the validset, and analyse  
**first, predict zero-nonzero-mask**

In [ ]:
#move checkpoint from gamma machine to here
cd checkpoint
scp -r endi.niu@10.171.36.214:/home/endi.niu/SIIM/checkpoint/nonzero_classifier_UNetResNet34_768_v1_seed1234/ nonzero_classifier_UNetResNet34_768_v1_seed1234
cd logging
scp -r endi.niu@10.171.36.214:/home/endi.niu/SIIM/logging/nonzero_classifier_UNetResNet34_768_v1_seed1234.log nonzero_classifier_UNetResNet34_768_v1_seed1234.log

In [ ]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
import pickle
import os
import logging
import time
from IPython.core.debugger import set_trace

import torch
import torch.nn as nn
import torch.nn.functional as F

from utils.utils import save_checkpoint, load_checkpoint, set_logger
from utils.gpu_utils import set_n_get_device

from model.model_unet_classify_zero import UNetResNet34, predict_proba
from dataset.dataset import prepare_trainset

from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

%matplotlib inline

In [ ]:
######### Config the training process #########
#device = set_n_get_device("0, 1, 2, 3", data_device_id="cuda:0")#0, 1, 2, 3, IMPORTANT: data_device_id is set to free gpu for storing the model, e.g."cuda:1"
MODEL = 'UNetResNet34'#'RESNET34', 'RESNET18', 'INCEPTION_V3', 'BNINCEPTION', 'SEResnet50'
#AUX_LOGITS = True#False, only for 'INCEPTION_V3'
print('====MODEL ACHITECTURE: %s===='%MODEL)

device = set_n_get_device("0,1", data_device_id="cuda:0")#0, 1, 2, 3, IMPORTANT: data_device_id is set to free gpu for storing the model, e.g."cuda:1"
multi_gpu = [0, 1]#use 2 gpus

SEED = 1234#5678#4567#3456#2345#1234
debug = False# if True, load 100 samples
IMG_SIZE = (256,1600)
BATCH_SIZE = 32
NUM_WORKERS = 24

In [ ]:
train_dl, val_dl = prepare_trainset(BATCH_SIZE, NUM_WORKERS, SEED, IMG_SIZE, debug)

In [ ]:
# y should be makeup
y_valid = []
for i, (images, masks) in enumerate(val_dl):
    #if i==10:
    #    break
    #truth = masks.to(device=device, dtype=torch.float)
    truth = (torch.sum(masks.reshape(masks.size()[0], masks.size()[1], -1), dim=2, keepdim=False)!=0)#.to(device=device, dtype=torch.float)
    y_valid.append(truth.numpy())
y_valid = np.concatenate(y_valid, axis=0)
y_valid.shape

In [ ]:
y_valid.mean()

In [ ]:
net = UNetResNet34(debug=False).cuda(device=device)

In [ ]:
checkpoint_path = '../checkpoint/nonzero_classifier_UNetResNet34_256x1600_v2_seed1234/best.pth.tar'
net, _ = load_checkpoint(checkpoint_path, net)

if multi_gpu is not None:
    net = nn.DataParallel(net, device_ids=multi_gpu)

In [ ]:
%%time
preds_valid = predict_proba(net, val_dl, device, multi_gpu=multi_gpu, mode='valid', tta=True)

In [ ]:
y_valid.shape, preds_valid.shape

In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix
import copy

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def cal_metric(logit, truth):
    #pred = sigmoid(logit.cpu().detach())
    pred = sigmoid(logit)
    #truth = truth.cpu().detach().numpy()
    ##
    THRESHOLD_candidate = [0.5]#np.arange(0.01, 0.99, 0.01)
    N = len(THRESHOLD_candidate)
    best_threshold = [0.5, 0.5, 0.5, 0.5]
    best_score = -1
    tn, fp, fn, tp, pos_percent = 0, 0, 0, 0, 0.0

    for ch in range(4):
        for i in range(N):
            THRESHOLD = copy.deepcopy(best_threshold)
            THRESHOLD[ch] = THRESHOLD_candidate[i]
            _pred = pred>THRESHOLD
            _pred, truth = _pred.reshape(-1, 1), truth.reshape(-1, 1)
            
            _tn, _fp, _fn, _tp = confusion_matrix(truth, _pred).ravel()
            _auc = round(roc_auc_score(truth, _pred), 5)
            if _tn+_fn==0:
                fn_rate = 9999
            else:
                fn_rate = round(_fn/(_tn+_fn), 5)
            _pos_percent = (_tp+_fp)/(_tp+_fp+_tn+_fn)
            
            if _auc > best_score:
                best_threshold = copy.deepcopy(THRESHOLD)
                best_score = _auc
                tn, fp, fn, tp, pos_percent = _tn, _fp, _fn, _tp, _pos_percent
    return best_threshold, best_score, tn, fp, fn, tp, pos_percent

In [ ]:
best_threshold, best_score, tn, fp, fn, tp, pos_percent = cal_metric(preds_valid, y_valid)
best_threshold, best_score, tn, fp, fn, tp, pos_percent

In [ ]:
best_threshold, best_score, tn, fp, fn, tp, pos_percent = cal_metric(preds_valid, y_valid)
best_threshold, best_score, tn, fp, fn, tp, pos_percent

In [ ]:
# THRESHOLD_candidate = np.arange(0.01, 0.99, 0.01)#np.arange(0.1, 1.0, 0.1)

# N = len(THRESHOLD_candidate)
# best_threshold = None
# best_score = 9999

# for i in tqdm_notebook(range(N)):
#     THRESHOLD = THRESHOLD_candidate[i]
#     auc, fn_rate, tn, fp, fn, tp = cal_metric(preds_valid, y_valid, THRESHOLD)
#     #predict positive proportion should be ~28%
#     pos_percent = (tp+fp)/(tp+fp+tn+fn)
#     if 0.2<pos_percent<0.28:
#         flag = 'Yes'
#         if fn_rate < best_score:
#             best_threshold = THRESHOLD
#             best_score = fn_rate
#     else:
#         flag = 'No'
#     print('[%s]THRESHOLD: %.2f, metric_score: %.5f, pos_percent: %.3f; tn, fp, fn, tp, auc: %d, %d, %d, %d, %.5f'%(flag, THRESHOLD, fn_rate, pos_percent, tn, fp, fn, tp, auc))


In [ ]:
# THRESHOLD = best_threshold
# THRESHOLD, best_score

In [ ]:
print(classification_report(y_valid, sigmoid(preds_valid)>best_threshold))

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def predict_mask(logit, best_threshold):
    """Transform each prediction into mask.
    input shape: (256, 256)
    """
    #pred mask 0-1 pixel-wise
    #n = logit.shape[0]
    #IMG_SIZE = logit.shape[-1] #256
    #EMPTY_THRESHOLD = 100.0*(IMG_SIZE/128.0)**2 #count of predicted mask pixles<threshold, predict as empty mask image
    #MASK_THRESHOLD = 0.22
    #logit = torch.sigmoid(torch.from_numpy(logit)).view(n, -1)
    #pred = (logit>MASK_THRESHOLD).long()
    #pred[pred.sum(dim=1) < EMPTY_THRESHOLD, ] = 0 #bug here, found it, the bug is input shape is (256, 256) not (16,256,256)
    logit = sigmoid(logit)#.reshape(n, -1)
    pred = (logit>best_threshold).astype(np.int)
    return pred

In [ ]:
## visualize predicted masks
start = 1
total = 19

fig=plt.figure(figsize=(15, 20))
cnt = 0
for idx, (images, masks) in enumerate(val_dl):
    if idx<start:
        continue
    labels = (torch.sum(masks.reshape(masks.size()[0], -1), dim=1, keepdim=True)!=0).int()
    for j in range(BATCH_SIZE):#BATCH_SIZE=8
        cnt+=1
        pred_mask = predict_mask(preds_valid[idx*BATCH_SIZE+j], best_threshold)
        #if pred_mask.float().mean()==0:
        #    continue
        ax = fig.add_subplot(5, 4, cnt)
        plt.imshow(images[j][0].numpy(), plt.cm.bone)
        if labels[j]==1:
            title = 'GT=1'
        else:
            title = 'GT=0'
        if pred_mask==1:
            title += ';PRED=1'
        else:
            title += ';PRED=0'
        plt.title(title)
        if cnt>total:
            break
    if cnt>total:
            break

In [ ]:
#100, 256, 334, 378, 547, 667, 916
#47, 173
plt.imshow(_logit[173], cmap='Reds')
#plt.imshow(_logit[256]>0.001, cmap='Reds')
#(_logit[100]>0.01).sum()

In [ ]:
#_logit = sigmoid(preds_valid_masks)#.reshape(n, -1)
_pred = (_logit>MASK_THRESHOLD).astype(np.int)
_pred_clf = (_pred.reshape(_pred.shape[0], -1).sum(axis=1)<EMPTY_THRESHOLD).astype(np.int)
#pred[pred_clf.reshape(-1,)==1, ] = 0

In [ ]:
#unet mask model
print(classification_report(y_valid[:, 0], 1-_pred_clf[:y_valid.shape[0]]))

In [ ]:
(1-_pred_clf[:y_valid.shape[0]]).mean()

In [ ]:
#classify zero-nonzero model
print(classification_report(y_valid, sigmoid(preds_valid)>THRESHOLD))

In [ ]:
(sigmoid(preds_valid)>THRESHOLD).mean()

In [ ]:
size = [8, 4, 64, 64]
n = size[0]//2

masks = torch.cat([torch.randint(0, 2, [n]+size[1:]).float(), 
                   torch.randint(0, 1, [n]+size[1:]).float()], dim=0)
truth = (torch.sum(masks.reshape(masks.size()[0], masks.size()[1], -1), dim=2, keepdim=False)!=0).float()
logit = torch.randn((size[0], size[1]))

logit.shape, truth.shape

In [ ]:
Loss_FUNC = torch.nn.BCEWithLogitsLoss()
Loss_FUNC(logit, truth)

In [ ]:
import numpy as np
import copy
from sklearn.metrics import confusion_matrix, roc_auc_score

In [ ]:
def metric(logit, truth):
    """
    AUC score as metric
    """
    pred = sigmoid(logit.cpu().detach().numpy())
    truth = truth.cpu().detach().numpy()
    ##
    THRESHOLD_candidate = np.arange(0.01, 0.99, 0.01)
    N = len(THRESHOLD_candidate)
    best_threshold = [0.01, 0.01, 0.01, 0.01]
    best_score = -1
    tn, fp, fn, tp, pos_percent = 0, 0, 0, 0, 0.0

    for ch in range(4):
        for i in range(N):
            THRESHOLD = copy.deepcopy(best_threshold)
            THRESHOLD[ch] = THRESHOLD_candidate[i]
            _pred = pred>THRESHOLD
            _pred, truth = _pred.reshape(-1, 1), truth.reshape(-1, 1)
            
            _tn, _fp, _fn, _tp = confusion_matrix(truth, _pred).ravel()
            _auc = round(roc_auc_score(truth, _pred), 5)
            if _tn+_fn==0:
                fn_rate = 9999
            else:
                fn_rate = round(_fn/(_tn+_fn), 5)
            _pos_percent = (_tp+_fp)/(_tp+_fp+_tn+_fn)
            
            if _auc > best_score:
                best_threshold = copy.deepcopy(THRESHOLD)
                best_score = _auc
                tn, fp, fn, tp, pos_percent = _tn, _fp, _fn, _tp, _pos_percent
    return np.round(best_threshold, 2), best_score, tn, fp, fn, tp, pos_percent

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
%%time
metric(logit, truth)